In [1]:
import random

# StringIO behaves like a file object 
from io import StringIO 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

import copy

%matplotlib inline 

In [2]:
def median(a,b,c):
    if a > b:
        if a < c:
            return a
        elif b > c:
            return b
        else:
            return c
    else:
        if a > c:
            return a
        elif b < c:
            return b
        else:
            return c

In [3]:
class CqkProblem:
    def __init__(self, r, n, d, a, b, low, up):
        self.n = n
        self.r = r
        self.d = list(d)
        self.a = list(a)
        self.b = list(b)
        self.low = list(low)
        self.up = list(up)

In [4]:
def generate_cqk_problem(n):
    d = []
    low = []
    up = []
    b = []
    a = []
    temp = 0
    lb = 0.0
    ub = 0.0
    lower = 10
    upper = 25
    r = 0

    for i in range(n):
        
        b.append(10 + 14*random.random())
        low.append(1 + 14*random.random())
        up.append(1 + 14*random.random())
        if low[i] > up[i]:
            temp = low[i]
            low[i] = up[i]
            up[i] = temp
        
        lb = lb + b[i]*low[i];
        ub = ub + b[i]*up[i];
        
        #Uncorrelated
        d.append(random.randint(10,25))
        a.append(random.randint(10,25))
        
    r = lb + (ub - lb)*0.7;
    
    return CqkProblem( r, n, d, a, b, low, up)

In [5]:

def initial_lambda(p, lamb):
    s0=0.0
    q0=0.0
    slopes = []
    for i in range(p.n):
        slopes.append((p.b[i]/p.d[i])*p.b[i])
        s0 = s0 + (p.a[i] * p.b[i]) / p.d[i]
        q0 = q0 + (p.b[i] * p.b[i]) / p.d[i]
    lamb = (p.r-s0)/q0
    return lamb, slopes

In [6]:
def phi_lambda(p,lamb,phi,deriv,slopes,r):
    deriv = 0.0
    phi = r * -1
    x = []
    
    for i in range(p.n):
        
        x.append( (p.b[i] * lamb + p.a[i])/p.d[i])

        if x[i] < p.low[i]:
            x[i] = p.low[i]
        elif x[i] > p.up[i]:
            x[i] = p.up[i]
        else:
            deriv = deriv + slopes[i];
        phi = phi + p.b[i] * x[i];
    return deriv, phi, x

In [7]:
MAX_IT = 20
INFINITO_NEGATIVO = -999999999;
INFINITO_POSITIVO = 999999999;

def newton(p):
    lambs = [] 
    phis = []
    derivs = []
    phi = 0
    lamb = 0
    alfa = INFINITO_NEGATIVO;
    beta = INFINITO_POSITIVO;
    phi_alfa = 0.0;
    phi_beta = 0.0;
    deriv = 0
    x = []
    r = p.r
    
    lamb, slopes = initial_lambda(p,lamb)
    deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
    lambs.append(lamb)
    derivs.append(deriv)
    phis.append(phi)
    it = 1
#     print(it, deriv, phi,lamb)
    negativo = False
    while phi != 0.0 and it <= MAX_IT:
        if phi > 0:
#             print("positivo")
            beta = lamb
            lambda_n = 0.0
            if deriv > 0.0:
                
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                if lambda_n > alfa:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_beta = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_left(p,lamb);
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
                
        else:
            if it == 1:
                negativo = True
#             print("negativo")
            alfa = lamb;
            lambda_n = 0.0;

            if deriv > 0.0:
                lambda_n = lamb - (phi/deriv)
                if abs(lambda_n - lamb) <= 0.00000000001:
                    phi = 0.0
                    break
                
                if lambda_n < beta:
                    lamb = lambda_n
                else:
#                     print("aqui")
                    phi_alfa = phi;
#                     lamb = secant(p,x,alfa,beta,phi_alfa,phi_beta,r);
#             if deriv == 0.0:
#                 lamb = breakpoint_to_the_right(p,lamb)
#                 if lamb <= INFINITO_NEGATIVO or lamb >= INFINITO_POSITIVO:
#                     break
        
        
        deriv, phi, x = phi_lambda(p,lamb,phi,deriv,slopes,r)
        it = it + 1
        lambs.append(lamb)
        derivs.append(deriv)
        phis.append(phi)
        
    if phi == 0.0:
        return it,lambs, derivs, phis,slopes
    elif alfa == beta:
        return -1,lambs, derivs, phis,slopes
    else:
        return -2,lambs, derivs, phis,slopes

In [8]:
def func_ty(p):
    t = np.arange(0, 15, 0.5).tolist()
    y = copy.deepcopy(t)
    for j in range(len(t)):
        lamb = t[j]

        soma = 0

        for i in range(n):
            soma = soma + p.b[i] * median(p.low[i], (p.b[i] *lamb + p.a[i]) / p.d[i], p.up[i])

        y[j] = soma
    return y

In [213]:
lista = []
for i in range(50000):
    n = 100
    p = generate_cqk_problem(n)
    it,lambs, derivs, phis,slopes = newton(p)
    soma_a = 0
    soma_b = 0
    soma_low = 0
    soma_d = 0
    soma_up = 0
    for i in range(n):
        soma_a += p.a[i]
        soma_b += p.b[i]
        soma_low += p.low[i]
        soma_d += p.d[i]
        soma_up += p.up[i]
    
    soma_a = soma_a/n
    soma_b = soma_b/n
    soma_low = soma_low/n
    soma_d = soma_d/n
    soma_up = soma_up/n
    r = p.r/n
    sum_slopes = sum(slopes)
    if it > 3:
#         l_rs = [soma_a, soma_b, soma_d, r,lambs[0],phis[0],derivs[0],sum_slopes,lambs[1],lambs[2],lambs[3],phis[1],derivs[1],lambs[-1]]
        l_rs = [soma_a, soma_b, soma_d, r,lambs[0],lambs[1],lambs[2],lambs[3],lambs[-1]]


        lista.append(l_rs)

In [218]:
np.savetxt('instance_test_100x50000_reduced.txt', lista, delimiter = ' ',newline='\n', fmt="%f")

In [282]:
c = ''
with open("instance_test_100x50000_reduced.txt", "r") as fd:
    c = StringIO(fd.read())

In [283]:
d = c.read()
c = StringIO(d) 
d = np.loadtxt(c) 
# feature_names = ['a','b','d','r','inicital_lamb','init_phi','init_deriv','sum_slopes','second_lambda','third_lamb','fourth_lamb','second_phi','second_deriv','final_lamb']
feature_names = ['a','b','d','r','inicital_lamb','second_lamb','third_lamb','fourth_lamb','final_lamb']



In [284]:
knapsack = {"data":d, "feature_names": feature_names}
dataset = pd.DataFrame(knapsack['data'], columns = knapsack['feature_names'])

In [337]:
# Coletando x e y

X = dataset.iloc[:,:-4]
y = dataset['second_lamb'].values

In [338]:
X

,a,b,d,r,inicital_lamb
0,17.01,16.547587,17.13,151.304258,7.198277
1,18.28,17.717724,17.45,159.796787,6.845233
2,17.07,16.485588,17.30,137.859952,6.581118
3,17.04,16.532460,17.68,150.831854,7.554698
4,17.52,17.225625,17.30,148.401486,6.747587
...,...,...,...,...,...
38995,17.58,16.595837,16.69,144.012571,6.625242
38996,17.39,17.073849,17.60,155.163036,7.366812
38997,16.82,17.057746,17.79,157.462754,7.558564
38998,17.79,16.299373,17.56,140.742248,7.083276


In [339]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [340]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [341]:
# Padronização
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [342]:
# Aplicando a padronização aos dados
X_train_p = scaler.transform(X_train)
X_test_p = scaler.transform(X_test)

In [343]:

# from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

In [344]:
# mlp = MLPClassifier(hidden_layer_sizes = (30,30,30))
regr = MLPRegressor(activation='relu',random_state=1, max_iter=5000).fit(X_train_p, y_train)

In [345]:
regr.fit(X_train_p, y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=5000,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=1, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [346]:
regr.predict(X_test_p[:10])

array([9.02409369, 8.8768658 , 8.87890581, 8.85232142, 8.34674954,
       8.92446777, 8.6031471 , 8.69599253, 8.16824764, 8.97678009])

In [348]:
regr.score(X_test_p, y_test)

0.5473658940168414

In [273]:
y_test[:10]

array([9.001176, 8.791612, 8.891414, 9.031551, 7.898873, 9.721935,
       8.776284, 9.219124, 9.045535, 9.628319])

In [29]:
dataset

,a,b,d,r,inicital_lamb,init_phi,init_deriv,sum_slopes,second_lambda,third_lamb,fourth_lamb,second_phi,second_deriv,final_lamb
0,17.563,17.137339,17.040,153.789637,6.861025,-10980.003494,6519.650825,19633.199608,8.545164,8.699830,8.702720,-819.579639,5299.040793,8.702723
1,17.693,16.860005,17.378,152.173745,7.145828,-10733.790555,6209.600581,18705.260923,8.874408,9.058557,9.062409,-939.632669,5102.561985,9.062409
2,17.427,17.091528,17.568,154.809150,7.161347,-12648.973495,7183.312077,19052.168439,8.922230,9.252729,9.263391,-1803.698408,5457.502063,9.263391
3,17.586,17.111238,17.412,155.648969,7.114978,-11665.257903,7289.355120,19248.063742,8.715292,8.917252,8.922372,-1172.620970,5806.202783,8.922369
4,17.582,17.216332,17.283,154.783176,6.994001,-10668.697650,6876.022142,19428.736025,8.545581,8.788623,8.794218,-1321.161360,5435.941733,8.794218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19989,17.411,17.016685,17.556,152.326709,7.111428,-11114.647829,7196.731251,18856.980521,8.655831,8.911568,8.912931,-1367.302879,5346.505145,8.912931
19990,17.353,16.815114,17.340,151.900359,7.205516,-10269.059269,7059.747514,18550.279684,8.660109,8.825953,8.829031,-971.013709,5854.988059,8.829031
19991,17.500,16.873401,17.507,152.081341,7.226951,-10496.290822,7170.427011,18524.373004,8.690782,8.927505,8.931318,-1337.699584,5650.902346,8.931323
19992,17.582,17.116986,17.458,149.919828,6.840590,-12707.758254,7378.410411,19187.950882,8.562879,8.892406,8.909531,-1769.309063,5369.248407,8.909558


In [2]:
n = 100
pp = generate_cqk_problem(n)
itt,lambss, derivss, phiss,slopess = newton(pp)

NameError: name 'generate_cqk_problem' is not defined